<a href="https://colab.research.google.com/github/LidiiaMelnyk95/FSU_Jena_scripts/blob/main/language_tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install --upgrade language_tool_python
!pip install simple_colors

In [ ]:
import language_tool_python
import simple_colors

In [ ]:
import glob
import os
import pandas as pd
data = pd.read_csv('/content/Validation_Commas.csv', sep = ',')

In [ ]:
data.head()

,Unnamed: 0.2,Unnamed: 0,Unnamed: 0.1,Comment,Commas,"Claudius (correct = 1, not correct = 0)",Claudius comments,"Tanya (correct = 1, not correct = 0)",Tanya comments,Unnamed: 9
0,0,0,499,"""Einfach mal nicht homosexuell sein, dann pass...",NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,2061,@ geht mir manchmal immer noch so beim Lesen a...,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,1562,Mhm. Nur die AFD redet über dieses Thema. Nur ...,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3,1772,Was meinste genau?,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4,1217,Warum wird eigentlich ganz am Anfang schon dur...,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
data['Comment'].values

array(['"Einfach mal nicht homosexuell sein, dann passiert sowas nicht" Merkste was?Das sucht man sich nicht aus. Es ist einfach so, und da hilft nur die Transition.',
       "@ geht mir manchmal immer noch so beim Lesen akademischer Lektüre.In der Regel wird's kontinuierlich besser, wenn man am Ball bleibt.",
       'Mhm. Nur die AFD redet über dieses Thema. Nur die AFD bringt es in den Bundestag... jaja',
       ..., 'Leider noch nicht.',
       '@Mii Woher würde man denn wissen, dass da ein biologischer Mann in der Umkleide ist? Starren Sie anderen Menschen in der Umkleide auf die Genitalien? Und was wäre so schlimm daran, wenn die Dame aus dem Video die Frauenumkleide nutzen würde? Ich fände es eher seltsam, wenn sie in die Männerumkleide käme.',
       '@ : "Was willst du mir sagen?"> Dass die Sprache mitunter bewusst zur Sexusdifferenzierung eingesetzt wird & das grammatikalische (Genus) dabei das biologische Geschlecht (Sexus) anzeigt - z.B.: Lehrer & Lehrerinnen bzw. Lehrer*inn

In [ ]:
data = data.drop_duplicates(subset = ['Comment'], inplace = False)
data.shape

(1397, 10)

In [ ]:
import pandas as pd
import string
import language_tool_python
from tqdm import tqdm

class LanguageToolProcessor:
    def __init__(self, data):
        self.data = data
        self.tool = language_tool_python.LanguageTool('de-DE')
        self.corrected_df = pd.DataFrame()
        self.punctuation_df = pd.DataFrame()

    def apply_casing_correction(self):
        for i, row in tqdm(self.data.iterrows(), total=len(self.data), desc="Applying Casing Correction"):
            for line in row['Comment'].split('/n'):
                checked_lines = []
                checked_lines_not_beginning = []
                checked = self.tool.check(line)
                for k in checked:
                    try:
                        if k.category == 'CASING':
                            if k.ruleId == 'UPPERCASE_SENTENCE_START':
                                line_1 = line.replace(k.matchedText, ('DETECT' + '[' + (k.matchedText) + ']'))
                                self.data.at[i, 'Comment'] = line_1
                                corrected_sentence = k.replacements[0] + k.sentence[len(k.matchedText):]
                                checked_lines.append(corrected_sentence)
                            else:
                                for replacement in k.replacements[:3]:
                                    corrected_sentence = k.sentence.split(k.matchedText)[0] + replacement + k.sentence.split(k.matchedText).pop()
                                    checked_lines_not_beginning.append(corrected_sentence)
                                    line_1 = line.replace(k.matchedText, ('DETECT' + '[' + (k.matchedText) + ']'))
                                    self.data.at[i, 'Comment'] = line_1
                        self.data.at[i, 'Uppercased'] = checked_lines
                        self.data.at[i, 'Casing not beginning'] = checked_lines_not_beginning
                    except TypeError:
                        pass

        self.data.to_csv('casing_corrected.csv', index=False)

    def apply_spelling_correction(self):
        for i, row in tqdm(self.data.iterrows(), total=len(self.data), desc="Applying Spelling Correction"):
            for line in row['Comment'].split('/n'):
                checked = self.tool.check(line)
                checked_lines = []
                for k in checked:
                    try:
                        if (k.category == 'TYPOS') or (k.category == 'TYPOGRAPHY'):
                            self.corrected_df.at[len(self.corrected_df), 'SPELLING'] = line
                        word = line[int(k.offset):int(k.offset) + int(k.length) + 1]
                        corrected_sentence = line.replace(word, (' DETECT' + '[' + word + ']'))
                        self.corrected_df.at[len(self.corrected_df) + 1, 'Comment'] = corrected_sentence
                        self.corrected_df.at[len(self.corrected_df), 'SPELLING'] = line
                        self.corrected_df.at[len(self.corrected_df), 'RULE ID'] = k.ruleId
                    except IndexError:
                        pass

        self.corrected_df.to_csv('spelling_corrected.csv', index=False)

    def apply_punctuation_correction(self):
        self.punctuation_df['RULE ID'] = ''
        for i, row in tqdm(self.data.iterrows(), total=len(self.data), desc="Applying Punctuation Correction"):
            for line in row['Comment'].split('/n'):
                checked = self.tool.check(line)
                for k in checked:
                    try:
                        if k.category == 'PUNCTUATION':
                            self.data.at[i, 'RULE ID'] = str(k).split()[str(k).split().index('Rule') + 2].rstrip(',')
                            self.punctuation_df.at[len(self.punctuation_df), 'Comment'] = line
                    except IndexError:
                        pass

        self.data.to_csv('punctuation_corrected.csv', index=False)
        self.punctuation_df.to_csv('punctuation_df.csv', index=False)

lt_processor = LanguageToolProcessor(data)



/.+?(?=abc)/

In [ ]:
lt_processor.apply_casing_correction()

casing_df = lt_processor.casing_df
casing_df['RULE ID'].value_counts()

UPPERCASE_SENTENCE_START       148
DE_CASE                        115
ETWAS_GUTES                     15
DOPPELPUNKT_GROSS                9
DE_DU_UPPER_LOWER                7
GROSSSCHREIBUNG_MAL              4
PRAEP_PLUS_VERB                  4
VERB_FEM_SUBST                   3
ZEIT_SEINES_LEBENS               3
ALS_ERSTES                       3
DAS_INFINITIV_VON                3
IM_SITZEN                        3
NOMEN_KLEIN                      3
IM_KLAREN                        2
KLEIN_NACH_PUNKT                 2
JA_NEIN                          2
DES_WEITEREN                     2
ZUR_ABBITTE_ETC                  2
AM_BESTEN                        2
ANS_KLEINES_NOMEN                2
IHR_EUCH                         1
DIENST_SUBST                     1
SUB_VER_KLEIN                    1
AUF_JEDEN_FALL                   1
WENN_UND_ABER                    1
GRUENDE_SUBST                    1
SUBSTANTIVIERUNG_NACH_DURCH      1
SUBSTANTIVIERTES_ADJEKTIV        1
BEDENKEN            

In [ ]:
lt_processor.apply_punctuation_correction()

punctuation_df = lt_processor.punctuation_df
punctuation_df['RULE ID'].value_counts()

INDIREKTE_FRAGE                91
SUBJUNKTION_KOMMA              51
DE_DOUBLE_PUNCTUATION          42
DOPPELTES_AUSRUFEZEICHEN       39
ICH_GLAUBE_FUER_EUCH           38
EINHEIT_LEERZEICHEN            29
FRAGEZEICHEN_STATT_PUNKT       24
SUBJUNKTION_KOMMA_2            15
XJAEHRIG                        9
UNPAIRED_BRACKETS               8
PUNKT_ENDE_DIREKTE_REDE         6
VERINF_DAS_DASS_SUB             6
ABKUERZUNG_FEHLENDE_PUNKTE      6
DOPPELTE_SATZZEICHEN            5
KOMMA_VOR_UND_ZWAR              5
KOMMA_VOR_WIE                   5
KOMMA_VOR_UND_ODER              3
PFEILE                          3
ZEICHENSETZUNG_DIREKTE_REDE     2
SONDERN_KOMMA                   2
SEHR_SEHR_GUT                   2
IN_UND_AUSWENDIG                1
MEINST_DU_DAS_KOMMA             1
KONJUNKTION_DASS_DAS            1
WEGEN_DEM_WAS_KOMMA             1
WENN_DOCH_KOMMA                 1
LEERZEICHEN_RECHENZEICHEN       1
Name: RULE ID, dtype: int64

In [ ]:
lt_processor.apply_spelling_correction()

spelling_df = lt_processor.spelling_df
spelling_df['RULE ID'].value_counts()
spelling_df['RULE ID'].value_counts()

GERMAN_SPELLER_RULE                        1838
DOPPELTES_LEERZEICHEN                       264
WHITESPACE_RULE                             254
COMMA_PARENTHESIS_WHITESPACE                 85
LEERZEICHEN_VOR_AUSRUFEZEICHEN_ETC           43
LEERZEICHEN_HINTER_DOPPELPUNKT               21
SAGT_RUFT                                    11
LEERZEICHEN_NACH_VOR_ANFUEHRUNGSZEICHEN       9
OLD_SPELLING                                  5
SONDER                                        4
WEISS                                         4
INS_BEZUG                                     2
ZUVIEL                                        2
GESETZT_GESETZ                                2
IM_UM                                         1
SO_SCHNELLE_WIE                               1
WAHR_NEHMEN                                   1
EMAIL                                         1
MIT_MIR                                       1
AKZENT_STATT_APOSTROPH                        1
LEERZEICHEN_UND                         